In [ ]:
!wget https://github.com/ashukid/hed-edge-detector/raw/refs/heads/master/hed_pretrained_bsds.caffemodel
!wget https://github.com/ashukid/hed-edge-detector/raw/refs/heads/master/deploy.prototxt

In [ ]:
!curl -O https://github.com/ashukid/hed-edge-detector/raw/refs/heads/master/hed_pretrained_bsds.caffemodel
!curl -O https://github.com/ashukid/hed-edge-detector/raw/refs/heads/master/deploy.prototxt

In [11]:
import os
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
BASEPATH = os.getcwd()

In [3]:
args = {
    "image": "image.jpg",
    "video": "SCARA_robot_plan-view.avi",
    "edge_detector": ""
}

In [4]:
class CropLayer(object):
  def __init__(self, params, blobs):
    # initialize our starting and ending (x, y)-coordinates of
    # the crop
    self.startX = 0
    self.startY = 0
    self.endX = 0
    self.endY = 0
  def getMemoryShapes(self, inputs):
    # the crop layer will receive two inputs -- we need to crop
    # the first input blob to match the shape of the second one,
    # keeping the batch size and number of channels
    (inputShape, targetShape) = (inputs[0], inputs[1])
    (batchSize, numChannels) = (inputShape[0], inputShape[1])
    (H, W) = (targetShape[2], targetShape[3])
    # compute the starting and ending crop coordinates
    self.startX = int((inputShape[3] - targetShape[3]) / 2)
    self.startY = int((inputShape[2] - targetShape[2]) / 2)
    self.endX = self.startX + W
    self.endY = self.startY + H
    # return the shape of the volume (we'll perform the actual
    # crop during the forward pass
    return [[batchSize, numChannels, H, W]]

  def forward(self, inputs):
    # use the derived (x, y)-coordinates to perform the crop
    return [inputs[0][:, :, self.startY:self.endY,
        self.startX:self.endX]]

In [5]:
# load our serialized edge detector from disk
print("[INFO] loading edge detector...")
protoPath = os.path.sep.join([BASEPATH, args["edge_detector"],
	"deploy.prototxt"])
modelPath = os.path.sep.join([BASEPATH, args["edge_detector"],
	"hed_pretrained_bsds.caffemodel"])
# net = cv2.dnn.readNetFromCaffe(r"C:\Users\obase\Documents\python_script\GRAMAI\HED\deploy.prototxt", r"C:\Users\obase\Documents\python_script\GRAMAI\HED\hed_pretrained_bsds.caffemodel")
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
# register our new layer with the model
cv2.dnn_registerLayer("Crop", CropLayer)

[INFO] loading edge detector...


In [6]:
# load the input image and grab its dimensions
image = cv2.imread(args["image"])
(H, W) = image.shape[:2]
# convert the image to grayscale, blur it, and perform Canny
# edge detection
print("[INFO] performing Canny edge detection...")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
canny = cv2.Canny(blurred, 30, 150)

[INFO] performing Canny edge detection...


In [7]:
# construct a blob out of the input image for the Holistically-Nested
# Edge Detector
blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(W, H),
	mean=(104.00698793, 116.66876762, 122.67891434),
	swapRB=False, crop=False)
# set the blob as the input to the network and perform a forward pass
# to compute the edges
print("[INFO] performing holistically-nested edge detection...")
net.setInput(blob)
hed = net.forward()
hed = cv2.resize(hed[0, 0], (W, H))
hed = (255 * hed).astype("uint8")
# show the output edge detection results for Canny and
# Holistically-Nested Edge Detection

[INFO] performing holistically-nested edge detection...


In [ ]:
plt.imshow(hed)

In [ ]:
cv2_imshow( image)
cv2_imshow( canny)

In [ ]:
hed = canny

In [ ]:
# prompt: generate code to perform hough circle detection on the image in variable hed

# Perform Hough Circle Transform on the HED output
# The parameters min_dist, param1, param2, min_radius, max_radius might need tuning
# depending on the image and the expected circles.
circles = cv2.HoughCircles(hed, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                           param1=50, param2=30, minRadius=5, maxRadius=50)

# Draw detected circles on the original image
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles:
        cv2.circle(image, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(image, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

# Show the image with detected circles
plt.imshow( image)

In [ ]:
# prompt: generate code to read the video file SCARA_robot_plan-view.avi, read frame by frame, apply canny edge detection, use hough circle and write another video file

# Read the video file
print("[INFO] reading video file...")
cap = cv2.VideoCapture(args["video"])

# Get video properties for writing the output video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (frame_width, frame_height))

[INFO] reading video file...


In [15]:

pbar = tqdm(range(total_frames),desc="")
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        pbar.set_description(f" {i} of {total_frames}")
        
        # Convert frame to grayscale for Canny and Hough Circle
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection
        canny_frame = cv2.Canny(gray_frame, 30, 150)

        # construct a blob out of the input frame for the Holistically-Nested
        # Edge Detector, set the blob, and perform a forward pass to
        # compute the edges
        blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(W, H),
            mean=(104.00698793, 116.66876762, 122.67891434),
            swapRB=False, crop=False)
        net.setInput(blob)
        hed = net.forward()
        hed = cv2.resize(hed[0, 0], (W, H))
        hed = (255 * hed).astype("uint8")

        # Perform Hough Circle Transform on the Canny output
        # Adjust parameters as needed
        # circles = cv2.HoughCircles(canny_frame, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
        #                            param1=100, param2=30, minRadius=4, maxRadius=55)
        circles = cv2.HoughCircles(hed, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                                   param1=100, param2=30, minRadius=4, maxRadius=55)        
        # Draw detected circles on the original frame
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                cv2.circle(frame, (x, y), r, (0, 255, 0), 4)
                cv2.rectangle(frame, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

        # Write the frame with detected circles to the output video
        out.write(frame)

        # Optional: Display the processed frame (can slow down processing)
        # cv2_imshow(frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
        pbar.update(1)
        i +=1
    else:
        break

# Release everything when job is finished
cap.release()
out.release()
# cv2.destroyAllWindows()

print("[INFO] video processing finished and saved as output_video.avi")

# Download the output video
# files.download('output_video.avi')

 364 of 365:   0%|          | 0/365 [56:06<?, ?it/s]

[INFO] video processing finished and saved as output_video.avi
